In [448]:
import pandas as pd

In [449]:
# Leo los archivos a utilizar. Solo las columnas de mi interés.
trainData = pd.read_csv('../Data/train.csv', usecols=['id', 'text', 'target'])
testData = pd.read_csv('../Data/test.csv', usecols=['id', 'text'])

In [450]:
trainData.head()

,id,text,target
0,1,Our Deeds are the Reason of this #earthquake M...,1
1,4,Forest fire near La Ronge Sask. Canada,1
2,5,All residents asked to 'shelter in place' are ...,1
3,6,"13,000 people receive #wildfires evacuation or...",1
4,7,Just got sent this photo from Ruby #Alaska as ...,1


In [451]:
testData.tail()

,id,text
3258,10861,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,Storm in RI worse than last hurricane. My city...
3260,10868,Green Line derailment in Chicago http://t.co/U...
3261,10874,MEG issues Hazardous Weather Outlook (HWO) htt...
3262,10875,#CityofCalgary has activated its Municipal Eme...


In [452]:
# Separo los textos de los tweets en palabras.

# Creo un nuevo dataframe con las palabras separadas.
splittedDF = pd.DataFrame(trainData['text'].str.split(' ').to_list(), index=trainData['id']).stack()

# Reestablezco el id.
splittedDF = splittedDF.reset_index([0, 'id'])

# Le pongo a las segunda columna un nombre descriptivo.
splittedDF.columns = ['id','word']

splittedDF.head()

,id,word
0,1,Our
1,1,Deeds
2,1,are
3,1,the
4,1,Reason


In [453]:
# Agrego la columna target haciendo un merge con el set de entrenamiento.
splittedDF = splittedDF.merge(trainData, left_on = 'id', right_on = 'id', how = 'left')

# Elimino la columna text y id.
del splittedDF['text']
del splittedDF['id']

splittedDF.head()

,word,target
0,Our,1
1,Deeds,1
2,are,1
3,the,1
4,Reason,1


In [454]:
# Agrupo las palabras y hago un promedio del target.
trainedDF = splittedDF.groupby('word').mean().reset_index()

trainedDF.sample(10)

,word,target
26053,lead,0.0
28521,redeemeth,0.0
5927,@meshnewsdesk,1.0
24281,http://t.co/qi8QDw5dFG,1.0
7205,Bistro,0.0
28250,pumpkin,0.0
30983,undeserving,1.0
25497,india:,0.0
815,#MeditationByMSG,0.0
31226,visit,0.8


Este dataframe sería mi algoritmo entrenado, ahora separo el test en sus palabras y luego hago un for sobre cada palabra, hago un merge con trainedDF y un mean de los targets para saber el valor final de la predicción.

In [455]:
# Separo los textos de los tweets en palabras.

# Creo un nuevo dataframe con las palabras separadas.
testSplitted = pd.DataFrame(testData['text'].str.split(' ').to_list(), index=testData['id']).stack()

# Reestablezco el id.
testSplitted = testSplitted.reset_index([0, 'id'])

# Le pongo a las segunda columna un nombre descriptivo.
testSplitted.columns = ['id','word']

testSplitted.head()

,id,word
0,0,Just
1,0,happened
2,0,a
3,0,terrible
4,0,car


In [456]:
# Mergeo el valor del promedio del target de las palabras en este data set.
testDF = testSplitted.merge(trainedDF, left_on = 'word', right_on = 'word', how = 'left')

# Elimino la columna de las palabras.
del testDF['word']

# Agrupo por id.
testDF = testDF.groupby('id').mean().reset_index()

testDF.head()

,id,target
0,0,0.518448
1,2,0.420480
2,3,0.486957
3,9,0.666667
4,11,0.797811


In [457]:
# Reestablezco el id.
testDF = testDF.set_index(['id'])

# Guardo el archivo de predicciones.
testDF.to_csv('test-regresion.csv')

In [458]:
# Paso los valores a 1 y 0 dependiendo si target > 0.5 o < 0.5.
testDF['target'] = testDF['target'].apply(lambda x: 1 if x >= 0.5 else 0)

testDF.head()

,target
id,
0,1
2,0
3,0
9,1
11,1


In [459]:
# Guardo el nuevo archivo de predicciones.
testDF.to_csv('test-clasification.csv')

Utilizando los dataframes creados con las palabras separadas por tweet con su peso (splittedDF) se hará un tunning del entrenamiento realizando los siguientes procedimientos:

1 - Se pasan todas las palabras a lowercase.

2 - Se toman los links como si fueran una sola palabra 'http'.

3 - Se quitan los '\n' encontrados en los textos y las palabras vacias.

4 - Se quitan de las palabras todos los caracteres que no sean letras.

5 - Se quitan los stop words.

En cada paso se hará un nuevo submit verificando los resultados obtenidos.

In [460]:
# 1 - Se pasan todas las palabras a lowercase.
lowercasedDF = splittedDF
lowercasedDF['word'] = lowercasedDF['word'].str.lower()
lowercasedTrainedDF = lowercasedDF.groupby('word').mean().reset_index()

lowercasedTestDF = testSplitted
lowercasedTestDF['word'] = lowercasedTestDF['word'].str.lower()

lowercasedTest = lowercasedTestDF.merge(lowercasedTrainedDF, left_on = 'word', right_on = 'word', how = 'left')
del lowercasedTest['word']
lowercasedTest = lowercasedTest.groupby('id').mean().reset_index()
lowercasedTest = lowercasedTest.set_index(['id'])
lowercasedTest['target'] = lowercasedTest['target'].apply(lambda x: 1 if x >= 0.5 else 0)

lowercasedTest.to_csv('test-1.csv')

In [461]:
# 2 - Se toman los links como si fueran una sola palabra 'http'.
httpDF = lowercasedDF
httpDF.loc[httpDF['word'].str.contains('http', case=True), 'word'] = 'http'
httpTrainedDF = httpDF.groupby('word').mean().reset_index()

httpTestDF = lowercasedTestDF
httpTestDF.loc[httpTestDF['word'].str.contains('http', case=True), 'word'] = 'http'

httpTest = httpTestDF.merge(httpTrainedDF, left_on = 'word', right_on = 'word', how = 'left')
del httpTest['word']
httpTest = httpTest.groupby('id').mean().reset_index()
httpTest = httpTest.set_index(['id'])
httpTest['target'] = httpTest['target'].apply(lambda x: 1 if x >= 0.5 else 0)

httpTest.to_csv('test-21.csv')

In [462]:
# 2 - Se toman los links como si fueran una sola palabra 'http'.
# Pero descartando el dataframe del punto 1.
httpDF2 = splittedDF
httpDF.loc[httpDF['word'].str.contains('http', case=True), 'word'] = 'http'
httpTrainedDF = httpDF.groupby('word').mean().reset_index()

httpTestDF = testSplitted
httpTestDF.loc[httpTestDF['word'].str.contains('http', case=True), 'word'] = 'http'

httpTest = httpTestDF.merge(httpTrainedDF, left_on = 'word', right_on = 'word', how = 'left')
del httpTest['word']
httpTest = httpTest.groupby('id').mean().reset_index()
httpTest = httpTest.set_index(['id'])
httpTest['target'] = httpTest['target'].apply(lambda x: 1 if x >= 0.5 else 0)

httpTest.to_csv('test-22.csv')